# Putting it all together

Here we show an example of doing live inference using the models that have been prepared in previous notebooks. 

[WORK IN PROGRESS] Please check back soon for updates

## Set up notebook environment

In [1]:
# Reload functions every time
%load_ext autoreload 
%autoreload 2

In [2]:
from pathlib import Path
import sys
import json
import pandas as pd

PROJECT_ROOT = Path().home() 
USER_ROOT = Path().home() / 'work'

sys.path.append(str(USER_ROOT / 'patientflow' / 'src' / 'patientflow'))
sys.path.append(str(USER_ROOT / 'patientflow' / 'functions'))

model_file_path = USER_ROOT / 'patientflow' / 'trained-models'
# model_file_path.mkdir(parents=True, exist_ok=True)
data_path = USER_ROOT / 'patientflow' / 'data-raw'
media_file_path = USER_ROOT / 'patientflow' / 'notebooks' / 'img'
# media_file_path.mkdir(parents=True, exist_ok=True)

In [108]:
model_file_path

PosixPath('/home/jovyan/work/ed-predictor/trained-models')

## Load parameters

In [3]:
uclh = True

In [4]:
from load_config import load_config_file

if uclh:
    config_path = Path(USER_ROOT / 'patientflow' / 'config-uclh.yaml')
else:
    config_path = Path(USER_ROOT / 'patientflow' / 'config.yaml')

params = load_config_file(config_path)

prediction_times = params[0]
start_training_set, start_validation_set, start_test_set, end_test_set = params[1:5]

x1, y1, x2, y2 = params[5:9]
prediction_window = params[9]
epsilon = params[10]
time_interval = params[11]



## Load data

In [10]:
import pandas as pd
from load_data_utils import set_file_locations
from ed_admissions_data_retrieval import ed_admissions_get_data


if uclh:
    data_path = USER_ROOT / 'ed-predictor' / 'data-raw'
    model_file_path = USER_ROOT / 'ed-predictor' / 'trained-models'
    visits_path, visits_csv_path, yta_path, yta_csv_path = set_file_locations(uclh, data_path, config_path)
else:
    visits_csv_path, yta_csv_path = set_file_locations(uclh, data_path)

visits = ed_admissions_get_data(visits_csv_path)

if visits.index.name != 'snapshot_id':
    visits = visits.set_index('snapshot_id')

visits['snapshot_date'] = pd.to_datetime(visits['snapshot_date']).dt.date
visits['consultation_sequence'] = visits['consultation_sequence'].apply(lambda x: () if x == [] else tuple(x))


In [19]:
from datetime import datetime, time

# Set seed
import numpy as np
np.random.seed(42)

# Randomly pick a prediction moment to do inference on
random_row = visits[visits.training_validation_test == 'test'].sample(n=1)
prediction_time = random_row.prediction_time.values[0]
prediction_date = random_row.snapshot_date.values[0]
prediction_moment = datetime.combine(prediction_date, datetime.min.time()).replace(hour=prediction_time[0], minute=prediction_time[1])

prediction_snapshots = visits[(visits.prediction_time == prediction_time) & \
            (visits.snapshot_date == prediction_date)]
prediction_snapshots

,snapshot_date,prediction_time,visit_number,elapsed_los,sex,age_on_arrival,arrival_method,current_location_type,total_locations_visited,num_obs,...,latest_lab_results_lac,latest_lab_results_na,latest_lab_results_pco2,latest_lab_results_ph,latest_lab_results_wcc,training_validation_test,final_sequence,is_admitted,random_number,specialty
snapshot_id,,,,,,,,,,,,,,,,,,,,,
159973,2031-11-19,"(12, 0)",145695.0,8400.0,F,40.0,NaN,sdec_waiting,3.0,16.0,...,NaN,NaN,NaN,NaN,NaN,test,[CON30404001],False,30751,NaN
160311,2031-11-19,"(12, 0)",146062.0,3780.0,F,37.0,NaN,sdec_waiting,2.0,0.0,...,NaN,NaN,NaN,NaN,NaN,test,[CON30404001],False,23918,NaN
160389,2031-11-19,"(12, 0)",146138.0,8820.0,F,24.0,NaN,sdec_waiting,3.0,18.0,...,NaN,NaN,NaN,NaN,NaN,test,[CON30404001],False,40665,NaN
160426,2031-11-19,"(12, 0)",146165.0,95609.0,F,89.0,Ambulance,sdec,4.0,128.0,...,1.6,138.0,3.60,7.474,9.41,test,[CON157],False,143567,NaN
160494,2031-11-19,"(12, 0)",146236.0,13680.0,M,36.0,NaN,sdec_waiting,2.0,21.0,...,1.4,138.0,6.61,7.395,2.55,test,[CON30404001],False,80068,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160736,2031-11-19,"(12, 0)",146461.0,4253.0,F,31.0,NaN,rat,2.0,13.0,...,NaN,NaN,NaN,NaN,NaN,test,[],False,78295,NaN
160737,2031-11-19,"(12, 0)",146462.0,4248.0,M,54.0,Walk-in,utc,2.0,14.0,...,NaN,NaN,NaN,NaN,NaN,test,[],False,92039,NaN
160738,2031-11-19,"(12, 0)",146463.0,4091.0,F,56.0,NaN,waiting,1.0,3.0,...,NaN,NaN,NaN,NaN,NaN,test,[],False,28266,NaN


## Generate predictions

The predictions for input into the spreadsheet output are generated by the create_predictions() function

In [120]:
from ed_admissions_realtime_preds import create_predictions
prediction_window_hrs = prediction_window/60

In [208]:
??create_predictions

Signature:
create_predictions(
    model_file_path: str,
    snapshot_datetime: datetime.datetime,
    prediction_snapshots: pandas.core.frame.DataFrame,
    specialties: List[str],
    prediction_window_hrs: float,
    cdf_cut_points: List[float],
    special_category_func: Callable[[Any], Any],
    special_category_dict: Dict[str, Any],
    x1: float,
    y1: float,
    x2: float,
    y2: float,
) -> Dict[str, Dict[str, List[int]]]
Docstring: <no docstring>
Source:   
def create_predictions(
    model_file_path: str,
    snapshot_datetime: datetime, 
    prediction_snapshots: pd.DataFrame,  
    specialties: List[str],
    prediction_window_hrs: float,
    cdf_cut_points: List[float],
    special_category_func: Callable[[Any], Any], 
    special_category_dict: Dict[str, Any],
    x1: float,
    y1: float,
    x2: float,
    y2: float,
) -> Dict[str, Dict[str, List[int]]]:  # [SpecialtyType, DemandPredictions]:
    
    # initialisation
    hour = snapshot_datetime.hour
    minute = s

In [209]:
child_dict = {
    'medical': 0.0,
    'surgical': 0.0,
    'haem/onc': 0.0,
    'paediatric': 1.0
}

# Function to determine if the patient is a child
# This can be customized to any complex logic necessary
if uclh:
    is_child_func = lambda row: row['age_on_arrival'] < 18 
else:
    is_child_func = lambda row: row['age_group'] == '0-17'

In [210]:
create_predictions(
    model_file_path = model_file_path,
    snapshot_datetime = prediction_moment,
    prediction_snapshots = prediction_snapshots,
    specialties = ['surgical', 'haem/onc', 'medical', 'paediatric'],
    prediction_window_hrs = prediction_window/60,
    cdf_cut_points =  [0.9, 0.7], 
    special_category_func = is_child_func,
    special_category_dict = child_dict, 
    x1 = x1,
    y1 = y1, 
    x2 = x2, 
    y2 = y2)

/home/jovyan/work/patientflow/functions/ed_admissions_realtime_preds.py:80: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prediction_snapshots['elapsed_los_hrs'] = prediction_snapshots['elapsed_los']/3600


{'surgical': {'in_ed': [1, 1], 'yet_to_arrive': [1, 2]},
 'haem/onc': {'in_ed': [0, 0], 'yet_to_arrive': [0, 1]},
 'medical': {'in_ed': [2, 3], 'yet_to_arrive': [3, 4]},
 'paediatric': {'in_ed': [0, 0], 'yet_to_arrive': [0, 1]}}